<a href="https://colab.research.google.com/github/IGieckI/AIExperiments/blob/main/A2/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import warnings
import pandas as pd
#import torch

#from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
#from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from huggingface_hub import login

## Setup and Data Loading

In [2]:
# Login with huggingface token to get access to some models
login()

In [12]:
# Download of the two dataset from the github repository
!wget -O "a2_test.csv" "https://raw.githubusercontent.com/nlp-unibo/nlp-course-material/refs/heads/main/2025-2026/Assignment%202/data/a2_test.csv"
!wget -O "demonstrations.csv" "https://raw.githubusercontent.com/nlp-unibo/nlp-course-material/refs/heads/main/2025-2026/Assignment%202/data/demonstrations.csv"

--2025-11-12 09:28:51--  https://raw.githubusercontent.com/nlp-unibo/nlp-course-material/refs/heads/main/2025-2026/Assignment%202/data/a2_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51789 (51K) [text/plain]
Saving to: ‘a2_test.csv’

a2_test.csv         100%[===================>]  50.58K  --.-KB/s    in 0.04s   

2025-11-12 09:28:51 (1.20 MB/s) - ‘a2_test.csv’ saved [51789/51789]

--2025-11-12 09:28:51--  https://raw.githubusercontent.com/nlp-unibo/nlp-course-material/refs/heads/main/2025-2026/Assignment%202/data/demonstrations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

In [19]:
# Load data into pandas DataFrames
test_df = pd.read_csv("a2_test.csv")
demonstrations_df = pd.read_csv("demonstrations.csv")

print("Test Set Label distribution:\n", test_df["label_category"].value_counts())
test_df.head()

Test Set Label distribution:
 label_category
derogation    60
prejudiced    60
animosity     60
not-sexist    60
threats       60
Name: count, dtype: int64


,rewire_id,text,label_category
0,sexism2022_english-11647,She may be worth what someone paid for her in ...,derogation
1,sexism2022_english-15883,The men who white knight for women eventually ...,prejudiced
2,sexism2022_english-12494,Two thoughts to keep in mind about women and A...,derogation
3,sexism2022_english-26,"Wimmin. No matter how good they have it, they'...",prejudiced
4,sexism2022_english-7475,"So just a few months ago, while Republican wom...",prejudiced


In [21]:
print("Demonstrations Label distribution\n", demonstrations_df["label_category"].value_counts())
demonstrations_df.head()

Demonstrations Label distribution
 label_category
derogation    200
threats       200
animosity     200
prejudiced    200
not-sexist    200
Name: count, dtype: int64


,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-7417,female egg donors are allowed to be uggo as fu...,sexist,derogation,2.1 descriptive attacks,train
1,sexism2022_english-10155,My girlfriend is mentally retarded. Or if you ...,sexist,derogation,2.1 descriptive attacks,test
2,sexism2022_english-15154,Lock her UP!!!!,sexist,threats,1.2 incitement and encouragement of harm,train
3,sexism2022_english-13387,So you want to live in a world where women are...,sexist,animosity,"3.1 casual use of gendered slurs, profanities,...",dev
4,sexism2022_english-2990,"Special episode of Loomy Does Politics, by pop...",sexist,prejudiced,4.2 supporting systemic discrimination against...,train


In [24]:
# Creating the ground truth labels from the test set into numeric format
label_mapping = {
    "not-sexist": 0,
    "threats": 1,
    "derogation": 2,
    "animosity": 3,
    "prejudiced": 4,
}

le = LabelEncoder()
le.fit(
    list(label_mapping.keys())
)
y_true = le.transform(test_df["label_category"])

# Creating also the inverse mapping numeric_id to labels_name
idx_to_label = {v: k for k, v in label_mapping.items()}
label_names = [
    idx_to_label[i] for i in sorted(idx_to_label)
]

# Get the list of texts to process
test_texts = test_df["text"].tolist()

print(f"Ground truth labels examples: {y_true[:5]}")
print(f"Label names for plots: {label_names}")

Ground truth labels examples: [1 3 1 3 3]
Label names for plots: ['not-sexist', 'threats', 'derogation', 'animosity', 'prejudiced']
